In [31]:

import pandas as pd
import numpy as np
import luigi
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime, date, time, timedelta
from functions import reduce_mem_usage

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import f1_score, classification_report, plot_confusion_matrix
from sklearn.metrics import precision_recall_curve, roc_curve, auc

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# from lightgbm import LGBMClassifier
#
# from imblearn.over_sampling import RandomOverSampler
# from collections import Counter
#
# from boruta import BorutaPy

import time

In [3]:
%matplotlib inline

warnings.simplefilter("ignore")
pd.set_option("display.max_columns", 999)

In [4]:
TRAIN_PATH = "data/train_merge.csv"
TEST_PATH = "data/test_merge.csv"

In [5]:
train = reduce_mem_usage(pd.read_csv(TRAIN_PATH))
test = reduce_mem_usage(pd.read_csv(TEST_PATH))

Memory usage of the dataframe is 1614.34 MB
Memory usage after optimization is: 807.17 MB
Decreased by 50.0%
Memory usage of the dataframe is 141.63 MB
Memory usage after optimization is: 70.82 MB
Decreased by 50.0%


In [6]:
train.head(20)

,id,vas_id,buy_time_x,target,buy_time_y,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,80,82,83,84,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252
0,540968,8.0,1537131600,0.0,1541970000,-31.559971,327.360901,-45.500786,274.753204,-50.918247,253.648209,49.15382,195.883896,-1.835267,-12.831245,-2.736328,56.284081,-2.558594,-66.187500,-77.500000,-0.000725,-0.016434,-0.107056,-5.417969,-3.177734,-13.937500,54.500000,-0.094238,-0.001733,-0.009331,-2.082031,0.200195,-0.009094,2.648438,0.785645,0.788574,-0.001884,-0.000023,-0.00003,62.593750,-0.065613,-0.700684,-192.026962,239.787094,-1005.230652,1245.017822,-89.504288,-119.724358,-3.511719,33.250000,-36.750000,-7.894531,-7.997875,-10.697456,-28.551693,-17.125000,-16.093750,-26.171875,-5.332031,1.041016,-3.746094,-1.670898,-0.001656,9.320312,15.117188,0.174805,-0.108154,0.028351,-0.017578,-0.165649,-0.028458,-0.044464,-0.301025,-0.554688,-0.036835,-0.130005,-2.783203,-2.607422,-5.390625,-4.023438,-2.824219,-10.703125,-1.2015,-0.998268,-0.203247,-0.248779,-0.2229,-0.134033,-0.030533,-0.125854,-0.096985,-0.679688,-0.626953,-0.691895,-0.506836,-0.185303,-0.598633,-0.000115,-0.250244,-0.348877,-0.828613,-42.275913,-3.949219,-0.252930,-0.318115,-2.291016,-3.447266,-0.040039,-9.406250,-0.212158,-11.953125,-1.019531,-1.473633,0.379150,-1006.426575,1242.881104,0.032013,0.196167,236.454514,848.953552,-0.343506,-0.089722,-0.278809,-0.433105,-0.024048,-89.211945,-119.674408,-208.886353,0.031921,-0.030457,0.104858,1.381836,5.316406,1.426758,-0.468506,-0.017288,-190.625,5.855469,7.277344,2.173828,5.062500,-1.276367,-0.020142,-0.042633,-11.476562,-59.375000,34.031250,-17.890625,0.138550,-0.317139,-0.007023,-0.143311,-14.414062,-0.212646,-0.019562,-0.000044,-0.000379,-2.548828,-0.26123,-0.536133,-0.061493,-0.152100,-0.002596,1.322266,-0.014542,-0.009506,-8.179688,35.750000,-14.320312,-40.812500,-5.960938,33.187500,-12.617188,-26.671875,-0.028854,-0.063232,-0.019196,-0.033783,-0.003149,-0.005184,-0.001431,-0.00189,-1.257812,-2.792969,-1.932617,-5.007812,-15.976562,-36.812500,-9.351562,-25.328125,-0.034576,-0.163208,-109.06250,0.533203,-1.928711,-0.623535,-0.228149,-0.251953,-0.000567,0.566406,-0.000708,-0.029205,-0.104675,-0.001357,0.039215,0.665527,-0.008995,-11953.712891,-45175.257812,-0.623047,-30.716053,-61790.156250,-0.243164,-42051.167969,-9239.707031,-2.107422,-0.000083,-0.623047,-0.00004,-0.620605,-0.012260,-0.107849,969818880.0,-49.21875,-4.405604e+08,-120441800.0,-8.632812,-1.464844,-33.302380,810.871033,270.250000,-0.364746,-0.133789,-0.209473,269.750000,-109.884567,364.308990,41.625000,939.000,-108.437500,-504.437317,-53.617977,-12.296875,-845.373840,-613.770813,-20.996269,-37.625000,-28.747725,4.167111,7.304688,-12.175933,21.546875,0.0
1,1454121,4.0,1531688400,0.0,1531083600,547.270020,238.430893,533.329224,274.803192,527.911743,253.698212,-16.08618,-65.076096,-6.783660,-30.006538,-2.736328,-4.007526,-2.558594,578.000000,566.500000,-0.000725,-0.016434,-0.107056,-5.417969,-3.177734,-13.937500,-10.742188,-0.094238,-0.001733,-0.009331,-2.082031,0.200195,-0.009094,-0.351807,-0.214355,-0.211548,-0.001884,-0.000023,-0.00003,-2.660156,-0.065613,-0.700684,-192.026962,-2764.330078,-1102.746338,-1661.583740,-89.504288,-119.724358,-11

Датасеты имеют две колонки даты - подключения услуги и некого другого действия (покупки сим карты?). Вероятно, имеет смысл сгенерировать новый признак как разницу между этими значениями. Условно, время с подключения сим до подключения услуги.

In [7]:
# train['interval'] = train['buy_time_y'] - train['buy_time_x']
# test['interval'] = test['buy_time_y'] - test['buy_time_x']

In [68]:
features = [f for f in train.columns if f not in ['target','id']]
len(features)

252

Разделим данные на X и y

In [9]:
X = train.drop('target', axis='columns')
y = train.target
X_valid = test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Разделим признаки на бинарные, категориальные и вещественные.

In [131]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("DataFrame не содердит следующие колонки: %s" % cols_error)

In [ ]:
class FeaturesGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, features_list):
        self.features_list = features_list

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # assert isinstance(X, pd.DataFrame)

        try:
            if 'interval' in self.features_list:
                X['interval'] = X['buy_time_y'] - X['buy_time_x']

            return X
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("DataFrame не содердит следующие колонки: %s" % cols_error)

In [ ]:
boolean_features = []
categorical_features = []
numeric_features = []
for col in X[features].fillna(0):
    val_count = len(X[col].unique())
    if val_count == 2:
        boolean_features.append(col)
    elif val_count <= 10:
        categorical_features.append(col)
    else:
        boolean_features.append(col)


In [ ]:
new_features_list = ['interval']

In [127]:
transform_pipe = Pipeline([
    ('cs', ColumnSelector(columns=features)),
    ('fg', FeaturesGenerator(features_list=['interval'])),
    ('fu', FeatureUnion()),
])
transform_pipe.transform(X_test)

TypeError: __init__() missing 1 required positional argument: 'transformer_list'

In [ ]:
from sklearn import set_config

set_config(display='diagram')

transform_pipe

In [ ]:
transformer_list=[
    ("numeric_features", make_pipeline(
        ColumnSelector(numeric_features),
        SimpleImputer(strategy="mean"),
        StandardScaler()
    )),
    ("categorical_features", make_pipeline(
        ColumnSelector(categorical_features),
        SimpleImputer(strategy="most_frequent"),
        OneHotEncoder(handle_unknown='ignore')
    )),
    ("boolean_features", make_pipeline(
        ColumnSelector(boolean_features),
    ))
]

In [ ]:
f_prep_pipeline = make_pipeline(
    ColumnSelector(columns=features),
    FeaturesGenerator(features_list=['interval']),
    FeatureUnion(transformer_list=transformer_list),
)

In [128]:
transform_pipe.transform(X_test).head()


,vas_id,buy_time_x,buy_time_y,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,80,82,83,84,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,interval,interval1
831968,6.0,1544389200,1544994000,370.850037,94.850891,356.909210,42.243202,351.491760,21.138210,451.563812,437.953888,-6.78366,-30.006538,-2.736328,462.092468,-1.008789,-66.1875,-77.5,-0.000725,-0.016434,-0.107056,-5.417969,-3.177734,-13.937500,455.250000,-0.094238,-0.001733,-0.009331,-2.082031,-0.799805,-0.009094,2.648438,0.785645,0.788574,-0.001884,-0.000023,-0.00003,-2.660156,-0.065613,-0.700684,357.973053,-2758.727539,-1186.765869,-1571.961670,-89.504288,-119.265373,-119.6875,-54.59375,-65.06250,-7.894531,-7.997875,-8.172410,-28.292337,-40.125000,-45.093750,-66.187500,-29.328125,-4.957031,-9.742188,-1.670898,-0.001656,-17.687500,0.117310,-0.265137,-0.408203,-0.091675,-0.237549,-0.295654,-0.028458,-0.044464,-0.301025,-0.554688,-0.036835,-0.130005,-2.783203,-1.046875,-3.830078,-2.462891,-2.824219,-10.703125,-1.2015,-0.998268,-0.203247,-0.248779,-0.2229,-0.134033,-0.030533,-0.125854,-0.096985,-0.679688,-0.626953,-0.691895,-0.506836,-0.185303,-0.598633,-0.000115,-0.250244,-0.348877,-0.828613,-42.275913,-3.949219,-0.25293,-0.318115,-2.291016,-3.447266,-0.040039,-9.40625,-0.212158,-11.953125,-1.019531,-2.472656,-5.621094,-1187.961670,-1574.098389,-0.078003,0.206177,-2762.060059,-2115.012207,-0.343506,-0.089722,-0.278809,-0.433105,-0.024048,-89.211945,-119.215424,-208.427368,-0.058075,-0.060455,0.404785,-5.617188,-14.679688,-3.574219,-0.468506,-0.417236,-190.625,-1.143555,-2.722656,-0.826172,-1.935547,-1.276367,-0.020142,-0.042633,-29.796875,-116.000,34.031250,-31.890625,-3.861328,-0.317139,-0.007023,-0.143311,-40.406250,-0.212646,-0.019562,-0.000044,-0.000379,0.451172,-0.261230,-0.536133,-0.061493,-0.152100,-0.002596,-4.679688,-0.014542,0.180542,-21.1875,-44.375,-25.3125,-52.000000,-25.96875,-54.65625,-13.617188,-30.828125,-0.028854,-0.063232,-0.019196,-0.033783,-0.003149,-0.005184,-0.001431,-0.00189,-1.257812,-2.792969,-1.932617,-5.007812,-30.984375,-65.12500,-16.359375,-35.312500,-0.034576,-0.163208,-109.0625,-0.466797,-2.929688,-0.623535,0.771973,-0.251953,-0.000567,-0.433838,-0.000708,-0.029205,-0.104675,-0.001357,0.039215,-0.334473,-0.008995,-11953.712891,-45175.257812,0.377197,-30.716053,-61790.156250,-0.243164,-42051.167969,-9239.707031,-2.107422,-0.000083,0.377197,-0.00004,0.379150,-0.01226,-0.107849,972324480.0,407.50000,-440560416.0,-1.204418e+08,7.648438,-1.464844,-33.302380,-265.128998,-39.781250,-0.364746,-0.133789,-0.209473,-35.343750,-109.884567,-876.691040,-5.367188,-247.125,-108.4375,-512.437317,-106.617981,-17.296875,-977.373840,-613.770813,-25.996269,-37.625,-133.747726,-25.832888,-0.694336,-12.175933,-0.456055,0.0,604800,604800
407305,1.0,1538946000,1531688400,276.760040,326.200897,262.819214,273.593201,257.401764,252.488205,-16.086180,-59.146095,-6.78366,-24.076538,-2.736328,-4.007526,-2.558594,188.0000,264.0,-0.000725,-0.016434,-0.107056,-5.417969,28.843750,-13.937500,-10.742188,-0.094238,-0.001733,-0.009331,40.093750,0.200195,-0.009094,-0.351807,-0.214355,-0.211548,-0.001884,-0.000023,-0.00003,-2.660156,-0.065613,23.703125,307.973053,3707.852539,2905.024170,802.828430,6899.501953,5479.75048

In [129]:
transform_pipe = Pipeline([
    ('cs', ColumnSelector(columns=features)),
    ('fg', FeaturesGenerator(features_list=['interval'])),
    ('fu', FeatureUnion(transformer_list=transformer_list)),
])
transform_pipe.fit(X_test).head()
# f_prep_pipeline.steps

SpecificationError: nested renamer is not supported

In [130]:
type(X_test)

pandas.core.frame.DataFrame

In [79]:
f_prep_pipeline.steps

[('featureunion',
  FeatureUnion(transformer_list=[('numeric_features',
                                  Pipeline(steps=[('columnselector',
                                                   ColumnSelector(columns=set())),
                                                  ('simpleimputer',
                                                   SimpleImputer()),
                                                  ('standardscaler',
                                                   StandardScaler())])),
                                 ('categorical_features',
                                  Pipeline(steps=[('columnselector',
                                                   ColumnSelector(columns={'131',
                                                                           '15',
                                                                           '154',
                                                                           '216',
                                           

In [87]:
n_features = f_prep_pipeline.fit(X_test).shape[1]

TypeError: fit() takes 1 positional argument but 2 were given

In [54]:
f_prep_pipeline.fit(X_train)

ValueError: at least one array or dtype is required